<a id="top"></a>
# MastMissions Introduction
***
## Learning Goals

By the end of this tutorial, you will:

- Understand how to query the MAST Archive for mission-specific observations.
- Run metadata queries based on coordinates, object names, or non-positional criteria.
- Use optional search parameters to refine query results.
- Find the clues to stop Dr. Nefarious' evil plot.

## Table of Contents
* [Introduction](#Introduction)
* [MastMissions Metadata Queries](#MastMissions-Metadata-Queries)
    * [Setting up for Mission-Specific Queries](#Setting-up-for-Mission-Specific-Queries) and [Exercise 1](#Exercise-1)
    * [Optional Search Parameters](#Optional-Search-Parameters)
    * [Query by Object](#Query-by-Object) and [Exercise 2](#Exercise-2)
    * [Query by Region](#Query-by-Region) and [Exercise 3](#Exercise-3)
    * [Query by Criteria](#Query-by-Criteria) and [Exercise 4](#Exercise-4)
* [Secret Message](#Secret-Message)
* [Additional Resources](#Additional-Resources)
* [Exercise Solutions](#Exercise-Solutions)

## Introduction
Welcome! This tutorial details how to use the `astroquery.mast` module to find mission-specific observations. We will be focusing on data from the [Hubble Space Telescope (HST)](https://science.nasa.gov/mission/hubble/). 

Today's data come from the [Mikulski Archive for Space Telescopes (MAST)](https://archive.stsci.edu/), which hosts publicly accessible data products from space telescopes including HST. The module `astroquery.mast` supports querying these data products, and its API is available [here](https://astroquery.readthedocs.io/en/latest/mast/mast.html). We will specifically focus on the MastMissions class `astroquery.mast.MastMissions` to query for observations and metadata from the HST.

## Imports
To run this notebook, we are importing:

- *astroquery.mast* to query MAST data
- *astropy.coordinates* to create coordinate objects

In [ ]:
from astroquery.mast import MastMissions
from astropy.coordinates import SkyCoord

***

## MastMissions Metadata Queries

### Setting up for Mission-Specific Queries

In order to query mission-specific MAST data, we need to define a few things. First, we need to create a MastMissions object with the `mission` set as `'hst'` since we are interested in Hubble data. By default, Its `service` is set to `'search'`, which is our use-case. Currently, `MastMissions` supports 4 mission collections:
- HST (Hubble Space Telescope)
- JWST (James Webb Space Telescope)
- Classy (COS Legacy Archive Spectroscopic SurveY, High-level Science Product collection)
- Ullyses (UV Legacy Library of Young Stars as Essential Standards, High-level Science Product collection)

Let's dive in:

In [ ]:
# Create a MastMissions object and assign the mission to 'hst' for Hubble Space Telescope data
hst_mission = MastMissions(mission='hst')

# Display the `mission` and `service` attributes for our hst_mission object
print("Current mission is:", hst_mission.mission)
print("Current service is:", hst_mission.service)

We can also setup some helpful references for ourselves for later. As we start making queries, it's helpful to know what columns we can expect (and later, what columns we could even filter on!). We can display the available available column names for a given mission by running `get_column_list`, as we will run below. This also displays the `data_type` and `description`.


In [ ]:
# Get the available column names for HST
hst_mission.get_column_list()

### Exercise 1

Now, let's practice setting up a MissionsMast class for the [James Webb Space Telescope (JWST)](https://webb.nasa.gov/)! This will be your first clue for this notebook's secret message.

In [ ]:
# # Create MastMissions object and assign mission to be the James Webb Space Telescope
# jwst_mission = ...  # Define your object!
# jwst_columns= ...  # Count the columns

### Optional Search Parameters

Before we begin our queries, it's important to know how we can refine our results with optional keyword arguments. The following parameters are available:

- `radius`: For positional searches only. Only return results within a certain distance from an object or set of coordinates. Default is 3 arcminutes. 
- `limit`: The maximum number of results to return. Default is 5000.
- `offset`: Skip the first ***n*** results. Useful for paging through results.
- `select_cols`: A list of columns to be returned in the response.

As we walk through different types of queries, we will see these parameters in action!

### Query by Object

Now, let's dive into our actual queries! We can use object names to query metadata by calling the `query_object` function.

To kick things off, let's see if there are observations of TRAPPIST-1, which is a red dwarf in the Aquarius constellation.

In [ ]:
# Query for TRAPPIST-1
results = hst_mission.query_object('TRAPPIST-1')

# Display the first 5 results
print(f'Total number of results: {len(results)}')
results[:5]

There were 1058 total results, meaning that 1058 HST datasets were targeting TRAPPIST-1. Now, let's try refining our search a bit more.

- Each dataset is associated with a celestial coordinate, given by `targ_ra` (right ascension) and `targ_dec` (declination). By default, the query returns all datasets that fall within 3 arcminutes from the object's coordinates. Let's set the `radius` parameter to be 1 arcminute instead.
- Say that we're not interested in the first 150 results. We can assign `offset` to skip a certain number of rows.
- By default, a subset of recommended columns are returned for each query. However, we can specify exactly which columns to return using the `select_cols` keyword argument. The `search_pos`, `sci_targname`, and `ang_sep` columns are included automatically.

In [ ]:
# Refining our query for TRAPPIST-1
results = hst_mission.query_object(
    'TRAPPIST-1',
    radius=1,  # Search within a 1 arcminute radius
    offset=150,  # Skip the first 150 results
    select_cols=['sci_data_set_name', 'sci_start_time'] # Select certain columns
)

# Display the first 5 results
print(f'Total number of results: {len(results)}')
results[:5]

### Exercise 2

Now it's your turn! Try querying the HST mission data for the Crab Nebula (Messier 1 or'M1'). Search within a radius of 1 arcminute, skip the first 20 results, and select the `sci_dataset_name` and `sci_instrument` columns.

This will be your second clue for this notebook's secret message.

In [ ]:
# # Query for M1
# m1_results = missions.query_object(...)  # Write your query!

# # Display the first 5 results
# print(f'Total number of results: {len(m1_results)}')
# m1_results[:5]

### Query by Region

You can also use a `mission`object to query by a region in the sky. To do so, you pass a set of coordinates into the method `query_region` and specify a radius to apply. The query will return any results within the radius of the given target, a process which is also known as a cone search.

Let's try this for the coordinates (210.80227, 54.34895).

In [ ]:
# Create coordinate object
coords = SkyCoord(210.80227, 54.34895, unit=('deg'))

# Query for results within 5 arcminutes of coords
results = hst_mission.query_region(coords, radius=5)

# Display results
print(f'Total number of results: {len(results)}')
results[:5]

958 HST datasets fall within our cone search. In other words, their target coordinates are within 5 arcminutes of the coordinates that we defined.

### Exercise 3
HST has observed a galaxy Mrk 848B, which is actually interacting galaxy pair currently undergoing a merger event. These two galaxies are colliding and will eventually merge together into one large galaxy. They have a right ascension of 229.52558 degrees and a declination of 42.745842 degrees. Use the `query_region` function to search for datasets within 30 arcminutes of Mrk 848B. Select the `sci_data_set_name`, `sci_actual_duration`, `sci_pi_last_name`, and `sci_central_wavelength` columns.

This will be your third clue for this notebook's secret message.

In [ ]:
# # Mrk 848B coordinates
# Mrk_848B = SkyCoord(_, _, unit=('deg'))  # Fill in with Mrk 848B's coordinates

# # Query for datasets around Mrk 848B
# results = hst_mission.query_region(...)  # Write your query!

# # Display the first 5 results
# print(f'Total number of results: {len(results)}')
# results[:5]

### Query by Criteria

In some cases, we may want to run queries with non-positional parameters. To do so, we can use the `query_criteria` function.

For any of our query functions, we can filter our results by the value of columns in the dataset.

Let's say that we only want observations from HST's Advanced Camera for Surveys (ACS) instrument, and that we only want datasets connected to program number 10895.

In [ ]:
# Query with column criteria
results = hst_mission.query_criteria(
    sci_instrume='ACS',
    sci_pep_id = 10895,
    select_cols=[
        'sci_data_set_name', 'sci_targname',
        'sci_instrume', 'sci_pep_id', 
        'sci_pi_last_name', 'sci_actual_duration',
    ]
)

# Display the first 5 results
print(f'Total number of results: {len(results)}')
results[:10]

To exclude and filter out a certain value from the results, we can prepend the value with `!`.

Let's run the same query as above, but this time, we will filter out datasets coming from the ACS instrument.

In [ ]:
# Filtered query, excluding NIRCam datasets
results = hst_mission.query_criteria(
    sci_pep_id=10895,
    sci_instrume='!ACS', # Exclude datasets from the ACS instrument
    select_cols=['fileSetName', 'instrume', 'exp_type', 'program', 'pi_name']
)

# Display the first 5 results
print(f'Total number of results: {len(results)}')
results[:5]

Wow! There is no data for this particular program taken with an instrument other than the ACS

We can also use wildcards for more advanced filtering. Let's use the original query from above, but we will add an aperature type filter for the High-Resolution Channel (HRC).

In [ ]:
# Query with column criteria
results = hst_mission.query_criteria(
    sci_instrume='ACS',
    sci_pep_id = 10895,
    sci_aper_1234 = '*HRC*',
    select_cols=[
        'sci_data_set_name', 'sci_targname',
        'sci_instrume', 'sci_pep_id', 
        'sci_pi_last_name', 'sci_actual_duration',
        'sci_aper_1234'
    ]
)
results

To filter by multiple values for a single column, we use a string of the values delimited by commas.

To illustrate this, we will use a slightly different query. We query for datasets that have the HRC aperature type filter taken with the ACS. We will add another filter to match three different last names for principal investigators (PIs). We will also exclude the `sci_pep_id` (10895) we had examined above to find new observations.

In [ ]:
# Filtered query with multiple values
results = hst_mission.query_criteria(
    sci_instrume='ACS',
    sci_aper_1234 = '*HRC*',
    sci_pep_id = '!10895',
    sci_pi_last_name = 'KALAS, NOLL, RIESS',
    select_cols=[
        'sci_data_set_name', 'sci_targname',
        'sci_instrume', 'sci_pep_id', 
        'sci_pi_last_name', 'sci_actual_duration',
        'sci_aper_1234',
    ]
)

# Display the first 10 results
print(f'Total number of results: {len(results)}')
results[:10]

For columns with numeric or date values, we can filter using comparison values:

- `<`: Return values less than or before the given number/date
- `>`: Return values greater than or after the given number/date
- `<=`: Return values less than or equal to the given number/date
- `>=`: Return values greater than or equal to the given number/date

As an example, let's write a query to return all datasets with an observation date before June 1, 1990.

In [ ]:
# Query using comparison operator
results = hst_mission.query_criteria(
    sci_start_time='<1990-06-01',  # Must be observed before June 1, 1990
    select_cols=[
        'sci_data_set_name', 'sci_targname',
        'sci_instrume', 'sci_pep_id', 
        'sci_pi_last_name', 'sci_actual_duration',
        'sci_aper_1234', 'sci_start_time'
    ]
)

# Display results
print(f'Total number of results: {len(results)}')
results[:10]

For numeric or date data types, we can also filter with ranges. This requires the following syntax: `'#..#'`.

Let's write a query that uses range syntax to return datasets that belong to a program number between 1150 and 1155. We will also select for exposure durations that are greater than or equal to 100 seconds.

In [ ]:
# Query using range operator
results = hst_mission.query_criteria(
    sci_pep_id='10890..10895', # Program number between 10890a and 10895
    sci_actual_duration='>100',  # Exposure duration is greater than or equal to 100 seconds
    select_cols=['sci_data_set_name', 'sci_pep_id', 'sci_actual_duration'])

# Display results
print(f'Total number of results: {len(results)}')
results[:10]

### Exercise 4
It's time to apply all that you've learned! Write a non-positional query based on the following:

- Instument is Wide Field Camera (WFC) or Wide Field and Planetary Camera 2 (WFPC2) 
- The observations should include those from the Planetary Chip Camera (PC1) and its variations
- Filters should not includes the FR868N filter
- Right ascension is between 70 and 75 degrees
- Program number is greater than 1200.
- Skip the first 50 entries.
- Select the following columns: `sci_data_set_name`, `sci_targname`, `sci_instrume`, `sci_spec_1234`, `sci_aper_1234`, `sci_ra`, `sci_pep_id`

This will be your fourth clue for this notebook's secret message.

In [ ]:
# # A non-positional query with column criteria
# results = hst_mission.query_criteria(...)  # Write your query here!

# # Display results
# print(f'Total number of results: {len(results)}')
# results

## Secret Message

To get your secret message from this notebook, use the answers to your examples above as follows:

1. Treat the number of JWST columns found in exercise 1 as answer 1.
2. Grab the row corresponding to answer 1 from the m1_results table you found for exercise 2. Return the 6th character of the angular separation (`ang_sep`) for this row. This is your answer 2.
3. Now, grab the row corresponding to answer 2 from the table you found for exercise 3. The `sci_actual_duration` from this row is your answer 3. 
4. Finally, take the row corresponding to answer 3 from the table you found for exercise 4. The secret message from this notebook is the first character of the `sci_aperture_1234`.

Use this with solutions from the other notebooks to stop Dr. Nefarious!

## Additional Resources

- [MAST HST Search Form](https://mast.stsci.edu/search/ui/#/hst)
- [MAST HST Search API](https://mast.stsci.edu/search/docs/?urls.primaryName=hst_api)
- [`astroquery.mast` Documentation for Mission-Specific Searches](https://astroquery.readthedocs.io/en/latest/mast/mast_missions.html#mission-specific-search-queries)

## Citations

If you use `astroquery` for published research, please cite the
authors. Follow these links for more information about citing `astroquery`:

* [Citing `astroquery`](https://github.com/astropy/astroquery/blob/main/astroquery/CITATION)

## About this Notebook
Let the world know who the author of this great tutorial is! If possible and appropriate, include a contact email address for users who might need support (for example, `archive@stsci.edu`). You should also include keywords and a last updated date in this section.

**Author(s):** Celia Parts, Sam Bianco <br>
**Keyword(s):** Tutorial, HST, Astroquery, MastMissions <br>
**First published:** June 2025 <br>

***
[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/style-guides/master/guides/images/stsci-logo.png" alt="Space Telescope Logo" width="200px"/> 